# Make a series of Chloropleth Maps showing average sentiment of covid tweets over time with slider
### input: average_tweet_sentiment csvs for each day of April
### output: plotly chloropleth map

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
import plotly.express as px


In [23]:
paths_list=[]
for i in range(13,19):
    if i>9:
        paths_list.append('average_sentiment_{}.csv'.format(i))
    else:
        paths_list.append('average_sentiment_0{}.csv'.format(i))

print(paths_list)

['average_sentiment_13.csv', 'average_sentiment_14.csv', 'average_sentiment_15.csv', 'average_sentiment_16.csv', 'average_sentiment_17.csv', 'average_sentiment_18.csv']


In [24]:
#https://plotly.com/python/subplots/#simple-subplot
#https://plotly.com/python/violin/#violin-plot-with-goviolin

fig = go.Figure()

for step in np.arange(len(paths_list)):
    average_state_sentiment = pd.read_csv(paths_list[step], engine='python')

    fig.add_trace(
        go.Choropleth(
        locations=average_state_sentiment['state'], # Spatial coordinates
        z = average_state_sentiment['sentiment'].astype(float), # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'Blues',
        reversescale= True,
        colorbar_title = "Average Sentiment",
        text=average_state_sentiment.apply(lambda row: f"{row['sentiment']}<br>{row['state']}", axis=1),
        hoverinfo="text",
        visible=False,
        name= '{}'.format(step),
        zmax=0.00,
        zmin=-0.25

    )
    )


fig.update_traces(hovertemplate='<extra></extra>', selector=dict(type='histogram'))

fig.update_layout(
    geo_scope='usa', # limite map scope to USA
)

fig.update_layout(height=500, width=700, hovermode='x unified')

fig.update_layout(title_text = 'Average Sentiment on COVID Tweets in the US, April 2020')
fig.data[0].visible = True


steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Average Sentiment on COVID Tweets in the US, {}/04/2020".format(str(i+1))}],
        label= "{}/04/2020".format(i+1)
    )

    step["args"][0]["visible"][i] = True
  # Toggle i'th, i+1th and i+2th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]
fig.update_layout(
    sliders=sliders
)


fig.show()
#fig.write_html("slider_chloropleth_test.html")



In [26]:
fig.write_html("chloropleth.html")
